## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-03-18-53-26 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...
1,2026-01-03-18-47-36 +0000,wapo,Live updates: Trump says U.S. will ‘run’ Venez...,https://www.washingtonpost.com/world/2026/01/0...
2,2026-01-03-18-47-36 +0000,wapo,Venezuela has yet to confirm if its vice presi...,https://www.washingtonpost.com
3,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. 'will run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...
4,2026-01-03-18-39-58 +0000,nyt,Venezuelans in South Florida Celebrate Maduro’...,https://www.nytimes.com/2026/01/03/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2398/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
3,trump,37
0,venezuela,28
8,maduro,28
37,new,18
48,fire,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
0,2026-01-03-18-53-26 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...,128
29,2026-01-03-17-01-01 +0000,nypost,Trump says US ‘not afraid of boots on the grou...,https://nypost.com/2026/01/03/world-news/trump...,119
10,2026-01-03-18-17-46 +0000,nypost,Trump brushes off concerns about legality of U...,https://nypost.com/2026/01/03/world-news/trump...,118
3,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. 'will run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...,118
49,2026-01-03-15-08-23 +0000,nyt,Trump Celebrates Capture of Nicolas Maduro in ...,https://www.nytimes.com/2026/01/03/world/ameri...,110


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
0,128,2026-01-03-18-53-26 +0000,nyt,Venezuela Live Updates: Trump Says U.S. Will ‘...,https://www.nytimes.com/live/2026/01/03/world/...
193,74,2026-01-02-19-35-00 +0000,wsj,"After Swiss New Year’s Fire, Parents Race to F...",https://www.wsj.com/world/europe/after-swiss-n...
125,54,2026-01-03-03-00-00 +0000,wsj,The massive welfare-fraud scandal in Minnesota...,https://www.wsj.com/politics/policy/minnesota-...
18,39,2026-01-03-17-50-54 +0000,nypost,"Venezuelan dictator appears blindfolded, handc...",https://nypost.com/2026/01/03/world-news/venez...
55,30,2026-01-03-14-53-38 +0000,bbc,Swiss open criminal case against managers of s...,https://www.bbc.com/news/articles/cjdjl303zxlo...
106,28,2026-01-03-06-45-38 +0000,nypost,"Trump says Venezuelan leader Nicolas Maduro, w...",https://nypost.com/2026/01/03/world-news/at-le...
135,28,2026-01-03-01-49-33 +0000,nypost,Coast Guard suspends search for 77-year-old cr...,https://nypost.com/2026/01/02/world-news/coast...
112,27,2026-01-03-05-09-32 +0000,nypost,DOJ official warns Seattle mayor to leave jour...,https://nypost.com/2026/01/03/us-news/doj-offi...
199,27,2026-01-02-19-01-02 +0000,nypost,Florida biz owner stunned Trump had time to ‘p...,https://nypost.com/2026/01/02/us-news/trump-go...
198,26,2026-01-02-19-13-20 +0000,nypost,Elon Musk’s AI chatbot Grok says it generated ...,https://nypost.com/2026/01/02/business/elon-mu...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
